In [2]:
import pandas as pd

In [3]:
data=pd.read_csv("Reviews.csv")

In [4]:
foodtext=data['Text']
print(type(foodtext))

<class 'pandas.core.series.Series'>


In [5]:
ft=foodtext.tolist()
ft[10]

"I don't know if it's the cactus or the tequila or just the unique combination of ingredients, but the flavour of this hot sauce makes it one of a kind!  We picked up a bottle once on a trip we were on and brought it back home with us and were totally blown away!  When we realized that we simply couldn't find it anywhere in our city we were bummed.<br /><br />Now, because of the magic of the internet, we have a case of the sauce and are ecstatic because of it.<br /><br />If you love hot sauce..I mean really love hot sauce, but don't want a sauce that tastelessly burns your throat, grab a bottle of Tequila Picante Gourmet de Inclan.  Just realize that once you taste it, you will never want to use any other sauce.<br /><br />Thank you for the personal, incredible service!"

In [6]:
#to remove the tags
import re

def cleanhtml(raw_html):
  cleanr = re.compile('<.*?>')
  cleantext = re.sub(cleanr, '', raw_html)
  return cleantext
cleanhtml(ft[10])

"I don't know if it's the cactus or the tequila or just the unique combination of ingredients, but the flavour of this hot sauce makes it one of a kind!  We picked up a bottle once on a trip we were on and brought it back home with us and were totally blown away!  When we realized that we simply couldn't find it anywhere in our city we were bummed.Now, because of the magic of the internet, we have a case of the sauce and are ecstatic because of it.If you love hot sauce..I mean really love hot sauce, but don't want a sauce that tastelessly burns your throat, grab a bottle of Tequila Picante Gourmet de Inclan.  Just realize that once you taste it, you will never want to use any other sauce.Thank you for the personal, incredible service!"

In [65]:
ft[15]

"My daughter loves twizzlers and this shipment of six pounds really hit the spot. It's exactly what you would expect...six packages of strawberry twizzlers."

In [8]:
from textblob import TextBlob

In [9]:
blob=TextBlob(ft[0])
blob.noun_phrases

WordList(['vitality', 'dog food products', 'good quality', 'labrador'])

In [10]:
noun_p=[]
for i in range(100):
    noun_p.append(TextBlob(cleanhtml(ft[i])).noun_phrases)
noun_p[15]

WordList(['strawberry twizzlers'])

In [11]:
flat_list = [item for sublist in noun_p for item in sublist]
flat_list

['vitality',
 'dog food products',
 'good quality',
 'labrador',
 'product',
 'jumbo salted peanuts',
 'jumbo',
 'pillowy citrus gelatin',
 'filberts',
 'tiny squares',
 'tiny mouthful',
 'c.s',
 'lewis',
 'lion',
 'witch',
 'wardrobe',
 'edmund',
 'brother',
 'sisters',
 'witch',
 'secret ingredient',
 'robitussin',
 'root beer extract',
 'cherry soda',
 'great taffy',
 'great price',
 'wide assortment',
 'yummy taffy',
 'delivery',
 'taffy lover',
 'wild hair',
 'root beer',
 'particular favorites',
 'saltwater taffy',
 'great flavors',
 'expensive version',
 'fralinger',
 'right',
 'wheatgrass',
 'rye',
 'healthy dog food',
 'small puppies',
 'dog eats',
 'unique combination',
 'hot sauce',
 'hot sauce..i',
 'hot sauce',
 'tequila picante gourmet',
 'inclan',
 'incredible service',
 'chubby guy',
 'by-product food',
 'skinny boy',
 'food sits',
 'chubby boy',
 'felidae platinum',
 'new bag',
 'new food',
 'similar reviews',
 'formula changes',
 'new food',
 'good flavor',
 'twizzler

In [63]:
import editdistance
import io
import itertools
import networkx as nx
import nltk
import os


def setup_environment():
    """Download required resources."""
    nltk.download('punkt')
    nltk.download('averaged_perceptron_tagger')
    print('Completed resource downloads.')


def filter_for_tags(tagged, tags=['NN', 'JJ', 'NNP']):
    """Apply syntactic filters based on POS tags."""
    return [item for item in tagged if item[1] in tags]


def normalize(tagged):
    """Return a list of tuples with the first item's periods removed."""
    return [(item[0].replace('.', ''), item[1]) for item in tagged]


def unique_everseen(iterable, key=None):
    """List unique elements in order of appearance.
    Examples:
        unique_everseen('AAAABBBCCDAABBB') --> A B C D
        unique_everseen('ABBCcAD', str.lower) --> A B C D
    """
    seen = set()
    seen_add = seen.add
    if key is None:
        for element in [x for x in iterable if x not in seen]:
            seen_add(element)
            yield element
    else:
        for element in iterable:
            k = key(element)
            if k not in seen:
                seen_add(k)
                yield element


def build_graph(nodes):
    """Return a networkx graph instance.
    :param nodes: List of hashables that represent the nodes of a graph.
    """
    gr = nx.Graph()  # initialize an undirected graph
    gr.add_nodes_from(nodes)
    nodePairs = list(itertools.combinations(nodes, 2))

    # add edges to the graph (weighted by Levenshtein distance)
    for pair in nodePairs:
        firstString = pair[0]
        secondString = pair[1]
        levDistance = editdistance.eval(firstString, secondString)
        gr.add_edge(firstString, secondString, weight=levDistance)

    return gr


def extract_key_phrases(text):
    """Return a set of key phrases.
    :param text: A string.
    """
    # tokenize the text using nltk
    word_tokens = nltk.word_tokenize(text)

    # assign POS tags to the words in the text
    tagged = nltk.pos_tag(word_tokens)
    textlist = [x[0] for x in tagged]

    tagged = filter_for_tags(tagged)
    tagged = normalize(tagged)

    unique_word_set = unique_everseen([x[0] for x in tagged])
    word_set_list = list(unique_word_set)

    # this will be used to determine adjacent words in order to construct
    # keyphrases with two words

    graph = build_graph(word_set_list)

    # pageRank - initial value of 1.0, error tolerance of 0,0001,
    calculated_page_rank = nx.pagerank(graph, weight='weight')

    # most important words in ascending order of importance
    keyphrases = sorted(calculated_page_rank, key=calculated_page_rank.get,
                        reverse=True)

    # the number of keyphrases returned will be relative to the size of the
    # text (a third of the number of vertices)
    one_third = len(word_set_list) // 3
    keyphrases = keyphrases[0:one_third + 1]

    # take keyphrases with multiple words into consideration as done in the
    # paper - if two words are adjacent in the text and are selected as
    # keywords, join them together
    modified_key_phrases = set([])
    # keeps track of individual keywords that have been joined to form a
    # keyphrase
    dealt_with = set([])
    i = 0
    j = 1
    while j < len(textlist):
        first = textlist[i]
        second = textlist[j]
        if first in keyphrases and second in keyphrases:
            keyphrase = first + ' ' + second
            modified_key_phrases.add(keyphrase)
            dealt_with.add(first)
            dealt_with.add(second)
        else:
            if first in keyphrases and first not in dealt_with:
                modified_key_phrases.add(first)

            # if this is the last word in the text, and it is a keyword, it
            # definitely has no chance of being a keyphrase at this point
            if j == len(textlist) - 1 and second in keyphrases and \
                    second not in dealt_with:
                modified_key_phrases.add(second)

        i = i + 1
        j = j + 1

    return modified_key_phrases

In [66]:
noun_tr=[]
for i in range(100):
    noun_tr.append(extract_key_phrases(cleanhtml(ft[i])))
noun_tr

[{'Labrador', 'Vitality', 'finicky', 'processed', 'several'},
 {'Jumbo', 'Peanuts', 'product', 'unsalted'},
 {'Brother',
  'Edmund',
  'Filberts',
  'Wardrobe',
  'confection',
  'familiar',
  'flavorful',
  'gelatin',
  'mouthful',
  'pillowy',
  'powdered'},
 {'Extract', 'Robitussin', 'addition', 'ingredient', 'medicinal'},
 {'Delivery', 'assortment', 'quick', 'taffy', 'yummy taffy'},
 {'complaint',
  'delightful',
  'enjoyable',
  'husband',
  'particular',
  'peppermint',
  'red/black licorice-flavored',
  'watermelon'},
 {'Fralinger', 'beach-themed', 'expensive version', 'saltwater'},
 {'chewy', 'satisfying'},
 {'Rye', 'Wheatgrass'},
 {'digestion', 'feeding', 'healthy', 'required'},
 {'Inclan',
  'Picante Gourmet',
  'Tequila Picante',
  'combination',
  'ecstatic',
  'flavour',
  'incredible service',
  'internet',
  'personal',
  'tequila'},
 {'by-product', 'chubby', 'protein-rich', 'skinny', 'stale', 'week', 'weight'},
 {'Felidae Platinum', 'bowls', 'different', 'food', 'shape'

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
vector=TfidfVectorizer()
vector.fit(flat_list)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [13]:
print(vector.vocabulary_)

{'vitality': 600, 'dog': 148, 'food': 194, 'products': 436, 'good': 221, 'quality': 440, 'labrador': 298, 'product': 435, 'jumbo': 286, 'salted': 470, 'peanuts': 398, 'pillowy': 408, 'citrus': 103, 'gelatin': 212, 'filberts': 187, 'tiny': 566, 'squares': 513, 'mouthful': 347, 'lewis': 308, 'lion': 311, 'witch': 620, 'wardrobe': 602, 'edmund': 160, 'brother': 68, 'sisters': 491, 'secret': 478, 'ingredient': 271, 'robitussin': 463, 'root': 465, 'beer': 49, 'extract': 173, 'cherry': 92, 'soda': 503, 'great': 227, 'taffy': 542, 'price': 430, 'wide': 618, 'assortment': 30, 'yummy': 629, 'delivery': 138, 'lover': 319, 'wild': 619, 'hair': 240, 'particular': 394, 'favorites': 184, 'saltwater': 471, 'flavors': 193, 'expensive': 172, 'version': 597, 'fralinger': 200, 'right': 462, 'wheatgrass': 614, 'rye': 468, 'healthy': 248, 'small': 498, 'puppies': 437, 'eats': 159, 'unique': 586, 'combination': 110, 'hot': 259, 'sauce': 473, 'tequila': 554, 'picante': 406, 'gourmet': 223, 'inclan': 269, 'in

In [14]:
print(vector.idf_)

[6.78689738 6.78689738 6.78689738 6.78689738 6.78689738 6.78689738
 6.78689738 6.78689738 6.78689738 6.78689738 6.78689738 6.78689738
 6.78689738 6.78689738 6.0937502  6.0937502  6.38143227 6.78689738
 6.78689738 5.08214929 6.78689738 6.0937502  6.0937502  5.68828509
 5.53413441 6.78689738 6.38143227 6.78689738 6.78689738 6.38143227
 6.78689738 6.78689738 6.78689738 6.78689738 6.38143227 6.78689738
 6.78689738 5.53413441 6.38143227 6.78689738 6.38143227 4.84098723
 6.78689738 6.78689738 6.78689738 6.78689738 5.87060665 6.38143227
 5.53413441 6.38143227 6.78689738 6.78689738 6.78689738 6.0937502
 6.78689738 6.78689738 6.78689738 6.78689738 6.78689738 6.78689738
 6.78689738 6.0937502  5.87060665 6.38143227 5.40060302 6.0937502
 6.38143227 6.78689738 6.78689738 4.99513791 6.78689738 6.78689738
 6.78689738 6.38143227 6.78689738 6.78689738 6.78689738 6.78689738
 6.78689738 6.78689738 6.38143227 5.53413441 6.38143227 6.78689738
 6.78689738 6.38143227 6.78689738 6.78689738 6.78689738 6.786897

In [15]:
from sklearn.feature_extraction.text import CountVectorizer
# 2. instantiate CountVectorizer (vectorizer)
vect = CountVectorizer()

In [16]:
vect.fit(flat_list)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [17]:
vect.get_feature_names()

['10',
 '10lbs',
 '1300watt',
 '16',
 '20',
 'abby',
 'abdominal',
 'according',
 'actual',
 'additionally',
 'aftertaste',
 'ages',
 'albanese',
 'alcohol',
 'all',
 'allergies',
 'allergy',
 'altura',
 'am',
 'amazon',
 'america',
 'amounts',
 'and',
 'apple',
 'apples',
 'arrhythmia',
 'arrived',
 'art',
 'artifial',
 'ass',
 'assortment',
 'at',
 'ate',
 'athlete',
 'awesome',
 'awful',
 'back',
 'bad',
 'bag',
 'baggie',
 'bags',
 'balance',
 'bariatric',
 'basenji',
 'batches',
 'be',
 'bean',
 'beans',
 'bears',
 'beer',
 'beware',
 'bichon',
 'big',
 'bike',
 'bitter',
 'black',
 'bloody',
 'boil',
 'boston',
 'bottle',
 'bough',
 'box',
 'boxes',
 'boy',
 'brand',
 'brands',
 'breakfast',
 'breaking',
 'brother',
 'brown',
 'brunch',
 'bucks',
 'bulldog',
 'buyer',
 'buying',
 'by',
 'ca',
 'calorie',
 'came',
 'camelbak',
 'candies',
 'candy',
 'cane',
 'canes',
 'cappuccino',
 'carb',
 'carbonation',
 'cast',
 'celiac',
 'chance',
 'changes',
 'cheap',
 'cherry',
 'chock',
 

In [18]:
# 4. transform training data into a 'document-term matrix'
simple_train_dtm = vect.transform(flat_list)
simple_train_dtm

<651x633 sparse matrix of type '<class 'numpy.int64'>'
	with 1271 stored elements in Compressed Sparse Row format>

In [19]:
simple_train_dtm.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [20]:
ar=pd.DataFrame(simple_train_dtm.toarray(), columns=vect.get_feature_names())

In [21]:
d_dict={}
len(flat_list[0])

8

In [22]:
for i in range(len(flat_list)):
    if(len(flat_list[i])>3):
        d_dict[flat_list[i]]=1
    else:
        d_dict[flat_list[i]]=0

In [23]:
d_dict

{'vitality': 1,
 'dog food products': 1,
 'good quality': 1,
 'labrador': 1,
 'product': 1,
 'jumbo salted peanuts': 1,
 'jumbo': 1,
 'pillowy citrus gelatin': 1,
 'filberts': 1,
 'tiny squares': 1,
 'tiny mouthful': 1,
 'c.s': 0,
 'lewis': 1,
 'lion': 1,
 'witch': 1,
 'wardrobe': 1,
 'edmund': 1,
 'brother': 1,
 'sisters': 1,
 'secret ingredient': 1,
 'robitussin': 1,
 'root beer extract': 1,
 'cherry soda': 1,
 'great taffy': 1,
 'great price': 1,
 'wide assortment': 1,
 'yummy taffy': 1,
 'delivery': 1,
 'taffy lover': 1,
 'wild hair': 1,
 'root beer': 1,
 'particular favorites': 1,
 'saltwater taffy': 1,
 'great flavors': 1,
 'expensive version': 1,
 'fralinger': 1,
 'right': 1,
 'wheatgrass': 1,
 'rye': 0,
 'healthy dog food': 1,
 'small puppies': 1,
 'dog eats': 1,
 'unique combination': 1,
 'hot sauce': 1,
 'hot sauce..i': 1,
 'tequila picante gourmet': 1,
 'inclan': 1,
 'incredible service': 1,
 'chubby guy': 1,
 'by-product food': 1,
 'skinny boy': 1,
 'food sits': 1,
 'chubby

In [24]:
dataset=pd.DataFrame(list(d_dict.items()),columns=['Terms','label'])
dataset.head()

,Terms,label
0,vitality,1
1,dog food products,1
2,good quality,1
3,labrador,1
4,product,1


In [25]:
X = dataset.Terms
y = dataset.label
print(X.shape)
print(y.shape)

(518,)
(518,)


In [26]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(388,)
(130,)
(388,)
(130,)


In [27]:
X_test.head

<bound method NDFrame.head of 485                         wet food
273                bitter aftertaste
420      food chopper/food processor
315                     water bottle
256                        cold milk
62                   zip lock baggie
267         stove-top popcorn popper
146              convenience product
362                       short time
364                  massive amounts
290                   grocery stores
223               good hot breakfast
411                    dissolve good
47                incredible service
162                          regular
31              particular favorites
506                  health problems
201                        small pot
65                    favorite candy
304                          imagine
439                 severe reactions
494         natural balance dog food
172                health food store
322                         gatorade
260                             greg
320                        high heat
107     

In [28]:
vect = CountVectorizer()

In [29]:
vect.fit(X_train)

# 4. transform training data
X_train_dtm = vect.transform(X_train)

In [30]:
X_train_dtm = vect.fit_transform(X_train)

In [31]:
X_train_dtm

<388x518 sparse matrix of type '<class 'numpy.int64'>'
	with 821 stored elements in Compressed Sparse Row format>

In [32]:
# transform testing data (using fitted vocabulary) into a document-term matrix
X_test_dtm = vect.transform(X_test)
X_test_dtm

<130x518 sparse matrix of type '<class 'numpy.int64'>'
	with 137 stored elements in Compressed Sparse Row format>

In [33]:
# 1. import
from sklearn.naive_bayes import MultinomialNB

# 2. instantiate a Multinomial Naive Bayes model
nb = MultinomialNB()

In [34]:
# 3. train the model 
# using X_train_dtm (timing it with an IPython "magic command")

%time nb.fit(X_train_dtm, y_train)

Wall time: 2 ms


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [35]:
# 4. make class predictions for X_test_dtm
y_pred_class = nb.predict(X_test_dtm)

In [36]:
# calculate accuracy of class predictions
from sklearn import metrics
metrics.accuracy_score(y_test, y_pred_class)

0.9615384615384616

In [37]:
# print the confusion matrix
metrics.confusion_matrix(y_test, y_pred_class)

array([[  0,   5],
       [  0, 125]], dtype=int64)

In [38]:
# print message text for the false positives (incorrectly classified as KP)
X_test[y_pred_class > y_test]

# alternative less elegant but easier to understand
# X_test[(y_pred_class==1) & (y_test==0)]

128    too
102    yum
314    pop
11     c.s
179    mmm
Name: Terms, dtype: object

In [39]:
# print message text for the false negatives (incorrectly classified as KP)

X_test[y_pred_class < y_test]
# alternative less elegant but easier to understand
# X_test[(y_pred_class=0) & (y_test=1)]

Series([], Name: Terms, dtype: object)

In [40]:
X_test[(y_pred_class==1) & (y_test==1)]

485                         wet food
273                bitter aftertaste
420      food chopper/food processor
315                     water bottle
256                        cold milk
62                   zip lock baggie
267         stove-top popcorn popper
146              convenience product
362                       short time
364                  massive amounts
290                   grocery stores
223               good hot breakfast
411                    dissolve good
47                incredible service
162                          regular
31              particular favorites
506                  health problems
201                        small pot
65                    favorite candy
304                          imagine
439                 severe reactions
494         natural balance dog food
172                health food store
322                         gatorade
260                             greg
320                        high heat
107                      unique size
3

In [41]:
# calculate predicted probabilities for X_test_dtm (poorly calibrated)

# Numpy Array with 2C
# left Column: probability class 0
# right C: probability class 1
# we only need the right column 
y_pred_prob = nb.predict_proba(X_test_dtm)[:, 1]
y_pred_prob

# Naive Bayes predicts very extreme probabilites, you should not take them at face value

array([0.99689649, 0.9742268 , 0.99948141, 0.99056295, 0.9896023 ,
       0.9742268 , 0.97825384, 0.99074691, 0.96773161, 0.96773161,
       0.95966731, 0.99304409, 0.99447332, 0.9742268 , 0.97825384,
       0.9742268 , 0.96773161, 0.96773161, 0.98683781, 0.9742268 ,
       0.9742268 , 0.99977179, 0.99883392, 0.9742268 , 0.9742268 ,
       0.98360123, 0.97825384, 0.96773161, 0.9742268 , 0.99020541,
       0.9896023 , 0.98360123, 0.9742268 , 0.9742268 , 0.99304409,
       0.99928209, 0.97825384, 0.9742268 , 0.98618429, 0.96773161,
       0.96773161, 0.99056295, 0.99264465, 0.9742268 , 0.97825384,
       0.99173283, 0.9742268 , 0.9742268 , 0.99304409, 0.95966731,
       0.9742268 , 0.99403186, 0.96773161, 0.96773161, 0.97825384,
       0.96773161, 0.9742268 , 0.96773161, 0.99650991, 0.97825384,
       0.98360123, 0.99342199, 0.99489628, 0.96773161, 0.96773161,
       0.98683781, 0.9742268 , 0.98360123, 0.98360123, 0.9742268 ,
       0.97825384, 0.9742268 , 0.9973909 , 0.96773161, 0.96773

In [42]:
# calculate AUC
metrics.roc_auc_score(y_test, y_pred_prob)

0.6719999999999999

In [43]:
flat_list

['vitality',
 'dog food products',
 'good quality',
 'labrador',
 'product',
 'jumbo salted peanuts',
 'jumbo',
 'pillowy citrus gelatin',
 'filberts',
 'tiny squares',
 'tiny mouthful',
 'c.s',
 'lewis',
 'lion',
 'witch',
 'wardrobe',
 'edmund',
 'brother',
 'sisters',
 'witch',
 'secret ingredient',
 'robitussin',
 'root beer extract',
 'cherry soda',
 'great taffy',
 'great price',
 'wide assortment',
 'yummy taffy',
 'delivery',
 'taffy lover',
 'wild hair',
 'root beer',
 'particular favorites',
 'saltwater taffy',
 'great flavors',
 'expensive version',
 'fralinger',
 'right',
 'wheatgrass',
 'rye',
 'healthy dog food',
 'small puppies',
 'dog eats',
 'unique combination',
 'hot sauce',
 'hot sauce..i',
 'hot sauce',
 'tequila picante gourmet',
 'inclan',
 'incredible service',
 'chubby guy',
 'by-product food',
 'skinny boy',
 'food sits',
 'chubby boy',
 'felidae platinum',
 'new bag',
 'new food',
 'similar reviews',
 'formula changes',
 'new food',
 'good flavor',
 'twizzler

In [44]:
ts_dict={}
for x in flat_list:
    ts_dict.update({x:flat_list.count(x)})
ts_dict

{'vitality': 1,
 'dog food products': 1,
 'good quality': 1,
 'labrador': 1,
 'product': 3,
 'jumbo salted peanuts': 1,
 'jumbo': 1,
 'pillowy citrus gelatin': 1,
 'filberts': 1,
 'tiny squares': 1,
 'tiny mouthful': 1,
 'c.s': 1,
 'lewis': 1,
 'lion': 1,
 'witch': 2,
 'wardrobe': 1,
 'edmund': 1,
 'brother': 1,
 'sisters': 1,
 'secret ingredient': 1,
 'robitussin': 1,
 'root beer extract': 1,
 'cherry soda': 1,
 'great taffy': 1,
 'great price': 2,
 'wide assortment': 1,
 'yummy taffy': 1,
 'delivery': 1,
 'taffy lover': 1,
 'wild hair': 1,
 'root beer': 1,
 'particular favorites': 1,
 'saltwater taffy': 1,
 'great flavors': 1,
 'expensive version': 1,
 'fralinger': 1,
 'right': 1,
 'wheatgrass': 1,
 'rye': 1,
 'healthy dog food': 1,
 'small puppies': 1,
 'dog eats': 1,
 'unique combination': 2,
 'hot sauce': 4,
 'hot sauce..i': 2,
 'tequila picante gourmet': 2,
 'inclan': 2,
 'incredible service': 2,
 'chubby guy': 1,
 'by-product food': 1,
 'skinny boy': 1,
 'food sits': 1,
 'chubby

In [45]:
from nltk import bigrams
from nltk import trigrams
from collections import Counter

In [46]:
bi_tokens = Counter(bigrams(flat_list))
tri_tokens = Counter(trigrams(flat_list))

In [47]:
bi_tokens

Counter({('vitality', 'dog food products'): 1,
         ('dog food products', 'good quality'): 1,
         ('good quality', 'labrador'): 1,
         ('labrador', 'product'): 1,
         ('product', 'jumbo salted peanuts'): 1,
         ('jumbo salted peanuts', 'jumbo'): 1,
         ('jumbo', 'pillowy citrus gelatin'): 1,
         ('pillowy citrus gelatin', 'filberts'): 1,
         ('filberts', 'tiny squares'): 1,
         ('tiny squares', 'tiny mouthful'): 1,
         ('tiny mouthful', 'c.s'): 1,
         ('c.s', 'lewis'): 1,
         ('lewis', 'lion'): 1,
         ('lion', 'witch'): 1,
         ('witch', 'wardrobe'): 1,
         ('wardrobe', 'edmund'): 1,
         ('edmund', 'brother'): 1,
         ('brother', 'sisters'): 1,
         ('sisters', 'witch'): 1,
         ('witch', 'secret ingredient'): 1,
         ('secret ingredient', 'robitussin'): 1,
         ('robitussin', 'root beer extract'): 1,
         ('root beer extract', 'cherry soda'): 1,
         ('cherry soda', 'great taffy')

In [48]:
matching = [s for s in ft if "dog food" in s]
matching

['I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than  most.',
 'This is a very healthy dog food. Good for their digestion. Also good for small puppies. My dog eats her required amount at every feeding.',
 'While my dogs like all of the flavors that we have tried of this dog food, for some reason their itching increased when I tried the lamb and rice. I have some very itchy dogs and am giving them a limited ingredient dog food to try to help. The duck and sweet potato cut down on the itching significantly, but when we tried lamb and rice they started itching more once again. I like Natural Balance for the quality ingredients.',
 'Awesome dog food. However, when given to my "Boston", who has severe reactions to some food ingredients; his itching increased to violent jumping out of bed 

In [49]:
strdata=str(matching)
strdata.count('dog food')

13221

In [50]:
def phrase_frequency(txt):
    matching = [s for s in ft if txt in s]
    strdata=str(matching)
    return strdata.count(txt)
phrase_frequency('dog food')

13221

In [51]:
compdata={}
for i in range(100):
    compdata[flat_list[i]]=phrase_frequency(flat_list[i])

In [52]:
compdata

{'vitality': 107,
 'dog food products': 53,
 'good quality': 2912,
 'labrador': 142,
 'product': 197254,
 'jumbo salted peanuts': 0,
 'jumbo': 228,
 'pillowy citrus gelatin': 1,
 'filberts': 29,
 'tiny squares': 19,
 'tiny mouthful': 1,
 'c.s': 3,
 'lewis': 0,
 'lion': 1694,
 'witch': 11349,
 'wardrobe': 18,
 'edmund': 0,
 'brother': 1529,
 'sisters': 184,
 'secret ingredient': 151,
 'robitussin': 1,
 'root beer extract': 2,
 'cherry soda': 67,
 'great taffy': 4,
 'great price': 4672,
 'wide assortment': 16,
 'yummy taffy': 1,
 'delivery': 7738,
 'taffy lover': 2,
 'wild hair': 2,
 'root beer': 852,
 'particular favorites': 5,
 'saltwater taffy': 6,
 'great flavors': 161,
 'expensive version': 22,
 'fralinger': 0,
 'right': 38633,
 'wheatgrass': 171,
 'rye': 1131,
 'healthy dog food': 88,
 'small puppies': 6,
 'dog eats': 269,
 'unique combination': 37,
 'hot sauce': 3256,
 'hot sauce..i': 1,
 'tequila picante gourmet': 0,
 'inclan': 0,
 'incredible service': 2,
 'chubby guy': 1,
 'by-

In [53]:
ft[10]

"I don't know if it's the cactus or the tequila or just the unique combination of ingredients, but the flavour of this hot sauce makes it one of a kind!  We picked up a bottle once on a trip we were on and brought it back home with us and were totally blown away!  When we realized that we simply couldn't find it anywhere in our city we were bummed.<br /><br />Now, because of the magic of the internet, we have a case of the sauce and are ecstatic because of it.<br /><br />If you love hot sauce..I mean really love hot sauce, but don't want a sauce that tastelessly burns your throat, grab a bottle of Tequila Picante Gourmet de Inclan.  Just realize that once you taste it, you will never want to use any other sauce.<br /><br />Thank you for the personal, incredible service!"

In [54]:
import math
def doc_freq(a):
    c=[]
    for i in range(100):
        if a in ft[i]:
            c.append(1)
    return math.log(100/(0.1+sum(c)),10)
    #return sum(c)
doc_freq('dog food')


0.9546770212133425

In [55]:
def pf_ipf(val):
    return phrase_frequency(val)*doc_freq(val)

pf_ipf('dog food products')
    

103.80618768661405

In [56]:
l1=list(compdata.keys())
m=[]
for x in l1:
    m.append(len(x))
max_len=max(m)
max_len


25

In [57]:

def phrase_position(x):
    return (math.sqrt(math.log(1+len(x),10)*math.log(1+max_len)))

In [58]:
phrase_position('dog')

1.400560449758195

In [59]:
kplist={}
for x in l1:
    kplist[x]=phrase_position(x)*pf_ipf(x)

In [60]:
kplist

{'vitality': 565.9999415864855,
 'dog food products': 209.92986384886282,
 'good quality': 9307.651062585885,
 'labrador': 751.140109395149,
 'product': 243266.35155166883,
 'jumbo salted peanuts': 0.0,
 'jumbo': 1089.10583727906,
 'pillowy citrus gelatin': 4.125480170391521,
 'filberts': 153.40185332717832,
 'tiny squares': 70.89481937104966,
 'tiny mouthful': 3.784825964240989,
 'c.s': 12.605044047823752,
 'lewis': 0.0,
 'lion': 7669.126551504379,
 'witch': 27261.937060872715,
 'wardrobe': 95.21494344445551,
 'edmund': 0.0,
 'brother': 7868.215168447235,
 'sisters': 946.8617338092159,
 'secret ingredient': 598.1020649278922,
 'robitussin': 5.525995034391016,
 'root beer extract': 12.133950886394041,
 'cherry soda': 246.0658494692967,
 'great taffy': 22.501445334843456,
 'great price': 14698.30309391033,
 'wide assortment': 62.07035236565312,
 'yummy taffy': 3.672624618944727,
 'delivery': 40931.84624295537,
 'taffy lover': 7.345249237889454,
 'wild hair': 7.0706500321317325,
 'root b

In [61]:
WantedOutput = sorted(kplist, key=lambda x : kplist[x],reverse=True) 

In [62]:
WantedOutput

['product',
 'inc',
 'right',
 'color',
 'delivery',
 'candy',
 'witch',
 'thank',
 'licorice',
 'great price',
 'strawberry',
 'according',
 'pounds',
 'candies',
 'hot sauce',
 'good flavor',
 'good quality',
 'brother',
 'lion',
 'reasonable price',
 'feet',
 'kg',
 'rye',
 'kosher',
 'record',
 'root beer',
 'new food',
 'favorite candy',
 'jumbo',
 'new bag',
 'sisters',
 'wheatgrass',
 'dog eats',
 'guilty pleasure',
 'labrador',
 'great flavors',
 'secret ingredient',
 'vitality',
 'hershey',
 'firms',
 'twizzler',
 'healthy dog food',
 'twizzlers',
 'records',
 'cherry soda',
 'dog food products',
 'cool place',
 'subsidiary',
 'filberts',
 'unique combination',
 'wardrobe',
 'july',
 'expensive version',
 'tiny squares',
 'zip lock baggie',
 'wide assortment',
 'similar reviews',
 'food sits',
 'record-breaking',
 'formula changes',
 'saltwater taffy',
 'small puppies',
 'great taffy',
 'guinness',
 'particular favorites',
 'strawberry twizzlers',
 'c.s',
 'root beer extract',